<center><font size=5>第22章 开发高效算法</font></center>

**小窍门**

<font color='dd0000'>二进制字符串转十进制</font>

In [4]:
public int binaryToDecimal(String binary){
    char[] b = binary.toCharArray();
    int res = 0;
    for(int i = 0; i < b.length; i++){
        if(b[i] == '0')
            res = res * 2 + 0;
        else if(b[i] == '1')
            res = res * 2 + 1;
    }
    return res;
}

String x = "1011010";
binaryToDecimal(x);

90

计算a的n次方:

In [1]:
// 方法1：将a乘n次：时间复杂度为n
public static int m1(int a, int n){
    int result = 1;
    for(int i = 1; i <= n; i++){
        result *= a;
    }
    return result;
}

// 方法2：将a每次进行平方(并处理非2的整次幂情况)：时间复杂度为log(n)
public static int m2(int a, int n){
    if(n == 1){
        return a;
    }else{
        int temp = m2(a, n / 2);
        if(n % 2 == 0)
            return temp *temp;
        else
            return a * temp * temp;
    }
}

In [2]:
m1(2,13);

8192

In [3]:
m2(2,13)

8192

**使用动态编程计算斐波那契数**

In [4]:
// 递归方法：复杂度为2的n次方

public static int fib(int index){
    if(index == 0)
        return 1;
    if(index == 1)
        return 1;
    return fib(index - 1) + fib(index - 2);
}

fib(6)

13

In [5]:
// 非递归方法：复杂度为O(n)

public static int fib2(int n){
    if(n < 2)
        return 1;
    
    int f0 = 1;
    int f1 = 1;
    int f2 = 0;
    for(int i = 2; i <= n; i++){
        f2 = f0 + f1;
        f0 = f1;
        f1 = f2;
    }
    return f2;
}

fib2(5)

8

上述非递归方法使用了一种称为动态编程的方法：通过解决子问题，然后将子问题的结果结合起来获得整个问题的解

**使用欧几里得算法求最大公约数**

In [6]:
// 暴力解法：假设m>n，则复杂度为O(n)

public static int gcd(int m, int n){
    int res = 1;
    for(int i = 2; i <= m && i <= n; i++){
        if(m % i ==0 && n % i == 0 )
            res = i;
    }
    return res;
}

gcd(45, 99);

9

In [7]:
// 欧几里得解法：最坏情况复杂度为O(log(n)

public static int euclidGcd(int m, int n){
    // 假设m > n
    if(n == 0)
        return m;
    int r = m % n;
    return euclidGcd(n, r);
}

euclidGcd(2525, 125)

25

**寻找素数的高效算法**

In [8]:
// 求不超过n的所有素数

public static void primeNumbers(int n){
    int i = 2;
    boolean isPrime = true;
    while(i <= n){
        int squareRoot = (int) Math.sqrt(i);
        for(int j = 2; j <= squareRoot; j++){
            if(i % j == 0){
                isPrime = false;
                break;
            }
        }
        if(isPrime){
            System.out.print(i + " ");
        }
        isPrime = true;
        i++;
    }
}

primeNumbers(17)

2 3 5 7 11 13 17 

**优化：**
没有必要对每个```i```来确切计算```Math.sqrt(i)```，只需要判断到所有小于n的完全平方数的根

如：4-9之间的数只需要判断到2 + 1 = 3，9-16之间的数只需要判断到3 + 1 = 4，16-25之间的数只需要判断到4 + 1 = 5...

In [9]:
public static void primeNumbers(int n){
    int i = 2;
    int squareRoot = 0;
    boolean isPrime = true;
    while(i <= n){
        
        if(i > squareRoot * squareRoot)
            squareRoot++;
        
        for(int j = 2; j <= squareRoot; j++){
            if(i % j == 0){
                isPrime = false;
                break;
            }
        }
        if(isPrime){
            System.out.print(i + " ");
        }
        isPrime = true;
        i++;
    }
}

primeNumbers(20)

2 3 5 7 11 13 17 19 

**继续优化：**确定***i***是否是素数，要检测*2,3,4...sqrt(i)*能否被i整除，实际上<font color='dd0000'>只需要检测从**2到sqrt(i)之间的素数**能否被i整除</font>

可以证明：如果***i***不是素数，那么必定存在一个**素数*p***，满足***i=pq***，且***p≤q***

In [10]:
// 在数组线性表中存储子问题的结果，之后再使用它们来检查新的数是否为一个素数(动态编程思想)

public static void primeNumbers(int n){
    int i = 2;
    int squareRoot = 0;
    boolean isPrime = true;
    List<Integer> primeList = new ArrayList<>();
    while(i <= n){
        
        if(i > squareRoot * squareRoot)
            squareRoot++;
        
        for(int j = 0; j < primeList.size() && primeList.get(j) <= squareRoot; j++){  // 比较的数不能超过当前i的squareRootl 
            if(i % primeList.get(j) == 0){    // 把当前数i和primeList内的每个数进行比较
                isPrime = false;
                break;
            }
        }
        if(isPrime){
            primeList.add(i);           //检查到一个素数，就把它放进primeList
            System.out.print(i + " ");
        }
        isPrime = true;
        i++;
    }
}

primeNumbers(20)

2 3 5 7 11 13 17 19 

<font color='dd0000'>**埃拉托色尼算法**</font>

① 初始化一个大小为***n***的布尔型数组***primes***，初始值所有元素为true

② 因为2的倍数不是素数，所以对于所有的***2 ≤ i ≤ n/2***，将***primes\[2i]***设为false

③ 因为3的倍数不是素数，所以对于所有的***3 ≤ i ≤ n/3***，将***primes\[3i]***设为false

④ 不需要考虑4，因为4的倍数是2的倍数，已经考虑过了，所有4的倍数都被置为false了

……

以此类推考虑素数2,3,5,7……的倍数

之后如果***primes[i]***的值仍为true，则***i***就是素数

In [2]:
public static void eratosthenes(int n){
    boolean[] primes = new boolean[n + 1]; // 下标是0 ~ n
    // 初始化为true
    for(int i = 0; i < primes.length; i++){
        primes[i] = true;
    }
    for(int k = 2; k <= n / k; k++){
        if(primes[k]){
            for(int i = k; i <= n / k; i++){
                primes[k * i] = false;
            }
        }
    }
    
    for(int i = 2; i < primes.length; i++){
        if(primes[i])
            System.out.print(i + " ");
    }
}

eratosthenes(20);

2 3 5 7 11 13 17 19 

**使用分治法找最近的点对**

In [51]:
class Point implements Comparable<Point>{
    int x;
    int y;
    
    public Point(int x, int y){
        this.x = x;
        this.y = y;
    }
    
    public int compareTo(Point p){
        if(this.x == p.x)
            return this.y - p.y;
        return this.x - p.x;
    }
    
    public String toString(){
        return "(" + this.x + "," + this.y + ")";
    }
}

In [3]:
// Point类测试

Point a = new Point(2,4);
Point b = new Point(3,5);
Point c = new Point(1,9);
List<Point> list = new ArrayList<>();
list.add(a);
list.add(b);
list.add(c);
System.out.println(list);
Collections.sort(list);
Iterator<Point> it = list.iterator();
while(it.hasNext()){
    System.out.print(it.next() + " ");
}

[(2,4), (3,5), (1,9)]
(1,9) (2,4) (3,5) 

In [4]:
// 求两点间距离

public static double getDistance(Point a, Point b){
    return Math.sqrt((a.x - b.x) * (a.x - b.x) + (a.y - b.y) * (a.y - b.y));
}

In [5]:
// 暴力解法：穷举每两个点间的距离，复杂度为O(n^2)

public static void minDistance(Point[] arr){
    Point p1 = arr[0];
    Point p2 = arr[1];
    double min = getDistance(p1, p2);
    for(int i = 0; i < arr.length; i++){
        for(int j = i + 1; j < arr.length; j++){
            double d = getDistance(arr[i], arr[j]);
            if(d < min){
                p1 = arr[i];
                p2 = arr[j];
                min = d;
            }
        }
    }
    System.out.print(p1 + " " + p2 + " " + "min = " + min);
}

In [10]:
// 分治法

public static void minDistance2(Point[] points){
    Point[] res = split(points, 0, points.length - 1);
    double min = getDistance(res[0], res[1]);
    System.out.print(res[0] + " " + res[1] + " " + "min = " + min);
}

private Point[] split(Point[] points, int left, int right){    // 返回值是有两个点的数组(即一对最近点)
   if(right - left <= 2){
       return bruteForce(points, left, right);
   }
    
    int mid = left + (right - left) / 2;
    Point[] leftPair = split(points, left, mid);
    Point[] rightPair = split(points, mid + 1, right);
    return merge(leftPair, rightPair, points, left, right, mid);
}

private Point[] merge(Point[] leftPair, Point[] rightPair, Point[] points, int left, int right, int mid){
    Point[] pair = new Point[2];
    double d1 = getDistance(leftPair[0], leftPair[1]);
    double d2 = getDistance(rightPair[0], rightPair[1]);
    double d = Math.min(d1, d2);
    if(d == d1){
        pair[0] = leftPair[0];
        pair[1] = leftPair[1];
    }else{
        pair[0] = rightPair[0];
        pair[1] = rightPair[1];
    }
    
    int midX = points[mid].x;
    List<Point> stripL = new ArrayList<>();
    List<Point> stripR = new ArrayList<>();
    for(int i = left; i <= right; i++){
        if(i <= mid && midX - points[i].x <= d){
            stripL.add(points[i]);
        } else if(i > mid && points[i].x - midX <= d){
            stripR.add(points[i]);
        }
    }
    
    for(Point p : stripL){
        for(Point q : stripR){
            if(Math.abs(p.y - q.y) <= d){
                if(getDistance(p, q) < d){
                    d = getDistance(p, q);
                    pair[0] = p;
                    pair[1] = q;
                }
            }
        }
    }
    return pair;
}

private Point[] bruteForce(Point[] points, int left, int right){
    Point[] pair = new Point[2];
    pair[0] = points[left];
    pair[1] = points[left + 1];
    double min = getDistance(pair[0], pair[1]);
    for(int i = left; i <= right; i++){
        for(int j = i + 1; j <= right; j++){
            double d = getDistance(points[i], points[j]);
            if(d < min){
                min = d;
                pair[0] = points[i];
                pair[1] = points[j];
            }
        }
    }
    return pair;
}

In [11]:
// Test

Point[] points = new Point[500];
for(int i = 0; i < points.length; i++){
    points[i] = new Point((int)(Math.random()*5000), (int)(Math.random()*1000));
}

minDistance(points);
System.out.println();
// 先排序
Arrays.sort(points);
minDistance2(points);

(3892,37) (3893,32) min = 5.0990195135927845
(3892,37) (3893,32) min = 5.0990195135927845

**使用回溯法解决八皇后问题**

In [28]:
class Solution{
    int[] board;
    List<String> oneSolution;
    List<List<String>> allSolution = new ArrayList<>();
    
    public List<List<String>> solveNQueens(int n){
        board = new int[n];
        queens(0, n);
        return allSolution;
    }

    public void queens(int i, int n){
        if(i == n){
            oneSolution = getResult(n);
            allSolution.add(oneSolution);
            return;
        }
        
        for(int j = 0; j < n; j++){
            board[i] = j;
            if(isOk(i, j, n)){
                queens(i + 1, n);
            }
            board[i] = 0;
        }
    }

    public boolean isOk(int i, int j, int n){
        int row = i - 1;
        int left = j - 1;
        int right = j + 1;
        while(row >= 0){
            if(board[row] == j || (left >= 0 && board[row] == left) || (right < n && board[row] == right)){
                return false;
            }
            row--;
            left--;
            right++;
        }
        return true;
    }
    
    public List<String> getResult(int n){
        List<String> sol = new ArrayList<>();
        String str = "";
        for(int i = 0; i < n; i++){
            str += ".";
        }
        for(int i = 0; i < board.length; i++){
            StringBuilder s = new StringBuilder(str);
            int index = board[i];
            s.setCharAt(index, 'Q');
            sol.add(s.toString());
        }
        return sol;
    }
}

In [37]:
Solution s = new Solution();
System.out.println(s.solveNQueens(4));

[[.Q.., ...Q, Q..., ..Q.], [..Q., Q..., ...Q, .Q..]]


**计算几何：寻找凸包**

<font color='dd0000'>卷包裹算法(gift-wrapping algirithm)</font>

![](giftwrapped.png)

⭐<font color='dd0000'>**判断点在有向直线左侧、右侧还是直线上的方法 ↓**</font>

In [59]:
// 判断一个点(x2, y2)在有向直线p0、p1的左侧、右侧还是直线上
public int pointPosition(int x0, int y0, int x1, int y1, int x2, int y2){
    return (x1 - x0) * (y2 - y0) - (x2 - x0) * (y1 - y0);
}

public List<Point> getConvexHull(List<Point> s){
    List<Point> h = new ArrayList<>();
    Point start = s.get(0);
    int minY = start.y;
    h.add(start);
    for(int i = 0; i < s.size(); i++){
        if(s.get(i).y < minY){
            start = s.get(i);
            minY = start.y;
            h.add(0, start);
        }else if(s.get(i).y == minY){
            if(s.get(i).x < start.x){
                start = s.get(i);
                h.add(0, start);
            }
        }
    }
    Point h0 = h.get(0);
    Point t0 = h0;
    Point t1;
    if(s.get(0) != t0)
        t1 = s.get(0);
    else
        t1 = s.get(1);
    int x0, y0, x1, y1, x2, y2;
    while(true){
        x0 = t0.x;
        y0 = t0.y;
        for(Point p : s){
            x1 = t1.x;
            y1 = t1.y;
            x2 = p.x;
            y2 = p.y;
            if(pointPosition(x0, y0, x1, y1, x2, y2) < 0){
                t1 = p;
                continue;
            }
        }
        if(!h.contains(t1))
            h.add(t1);
        if(t1 == h0)
            break;
        t0 = t1;
        if(s.get(0) != t0)
            t1 = s.get(0);
        else
            t1 = s.get(1);
    }
    return h;
}

In [60]:
Point[] points = new Point[15];
for(int i = 0; i < points.length; i++){
    points[i] = new Point((int)(Math.random()*10), (int)(Math.random()*10));
}
List<Point> s = Arrays.asList(points);
System.out.println(s);
getConvexHull(s);

[(9,6), (4,9), (6,2), (8,3), (5,1), (7,6), (0,0), (3,6), (1,0), (8,0), (8,5), (2,1), (3,3), (1,9), (0,8)]


[(0,0), (5,1), (6,2), (9,6), (1,0), (8,0), (4,9), (1,9), (0,8)]

<font color='dd0000'>格雷厄姆算法(Graham algorithm)</font>

![](graham.png)